In [ ]:
!pip install -U transformers albumentations==1.4.0 numpy==1.24.4 omegaconf==2.3.0 opencv-python==4.11.0.86 opencv-python-headless==4.5.5.64 pillow==9.3.0 timm==0.5.4 torch==2.1.0 torchvision==0.16.0 transformers==4.47.0 accelerate==1.6.0 pymupdf==1.26

  Using cached transformers-4.52.4-py3-none-any.whl.metadata (38 kB)


Model page: https://huggingface.co/ByteDance/Dolphin

⚠️ If the generated code snippets do not work, please open an issue on either the [model repo](https://huggingface.co/ByteDance/Dolphin)
			and/or on [huggingface.js](https://github.com/huggingface/huggingface.js/blob/main/packages/tasks/src/model-libraries-snippets.ts) 🙏

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("image-text-to-text", model="ByteDance/Dolphin")

config.json:   0%|          | 0.00/4.85k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/796M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/4.05M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.86M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/277 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/478 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
"""
Copyright (c) 2025 Bytedance Ltd. and/or its affiliates
SPDX-License-Identifier: MIT
"""

import re
import base64
from typing import List, Dict, Any, Optional


"""
    Example input:
    [
        {"label": "tab", "bbox": [0.176, 0.74, 0.824, 0.82], "text": "<table><tr><td></td><td>HellaSwag</td><td>Obqa</td><td>WinoGrande</td><td>ARC-c</td><td>ARC-e</td><td>boolq</td><td>piqa</td><td>Avg</td></tr><tr><td>OPT-1.3B</td><td>53.65</td><td>33.40</td><td>59.59</td><td>29.44</td><td>50.80</td><td>60.83</td><td>72.36</td><td>51.44</td></tr><tr><td>Pythia-1.0B</td><td>47.16</td><td>31.40</td><td>53.43</td><td>27.05</td><td>48.99</td><td>57.83</td><td>69.21</td><td>48.30</td></tr><tr><td>Pythia-1.4B</td><td>52.01</td><td>33.20</td><td>57.38</td><td>28.50</td><td>54.00</td><td>63.27</td><td>70.95</td><td>51.33</td></tr><tr><td>TinyLlama-1.1B</td><td>59.20</td><td>36.00</td><td>59.12</td><td>30.10</td><td>55.25</td><td>57.83</td><td>73.29</td><td>52.99</td></tr></table>", "reading_order": 6},
        {"label": "cap", "bbox": [0.28, 0.729, 0.711, 0.74], "text": "Table 2: Zero-shot performance on commonsense reasoning tasks", "reading_order": 7},
        {"label": "para", "bbox": [0.176, 0.848, 0.826, 0.873], "text": "We of performance during training We tracked the accuracy of TinyLlama on common-\nsense reasoning benchmarks during its pre-training, as shown in Fig. 2 . Generally, the performance of", "reading_order": 8},
        {"label": "fnote", "bbox": [0.176, 0.88, 0.824, 0.912], "text": "${ }^{4}$ Due to a bug in the config file, the learning rate did not decrease immediately after warmup and remained at\nthe maximum value for several steps before we fixed this.", "reading_order": 9},
        {"label": "foot", "bbox": [0.496, 0.939, 0.501, 0.95], "text": "14", "reading_order": 10}
    ]
"""


def extract_table_from_html(html_string):
    """Extract and clean table tags from HTML string"""
    try:
        table_pattern = re.compile(r'<table.*?>.*?</table>', re.DOTALL)
        tables = table_pattern.findall(html_string)
        tables = [re.sub(r'<table[^>]*>', '<table>', table) for table in tables]
        return '\n'.join(tables)
    except Exception as e:
        print(f"extract_table_from_html error: {str(e)}")
        return f"<table><tr><td>Error extracting table: {str(e)}</td></tr></table>"


class MarkdownConverter:
    """Convert structured recognition results to Markdown format"""

    def __init__(self):
        # Define heading levels for different section types
        self.heading_levels = {
            'title': '#',
            'sec': '##',
            'sub_sec': '###'
        }

        # Define which labels need special handling
        self.special_labels = {
            'tab', 'fig', 'title', 'sec', 'sub_sec',
            'list', 'formula', 'reference', 'alg'
        }

    def try_remove_newline(self, text: str) -> str:
        try:
            # Preprocess text to handle line breaks
            text = text.strip()
            text = text.replace('-\n', '')

            # Handle Chinese text line breaks
            def is_chinese(char):
                return '\u4e00' <= char <= '\u9fff'

            lines = text.split('\n')
            processed_lines = []

            # Process all lines except the last one
            for i in range(len(lines)-1):
                current_line = lines[i].strip()
                next_line = lines[i+1].strip()

                # Always add the current line, but determine if we need a newline
                if current_line:  # If current line is not empty
                    if next_line:  # If next line is not empty
                        # For Chinese text handling
                        if is_chinese(current_line[-1]) and is_chinese(next_line[0]):
                            processed_lines.append(current_line)
                        else:
                            processed_lines.append(current_line + ' ')
                    else:
                        # Next line is empty, add current line with newline
                        processed_lines.append(current_line + '\n')
                else:
                    # Current line is empty, add an empty line
                    processed_lines.append('\n')

            # Add the last line
            if lines and lines[-1].strip():
                processed_lines.append(lines[-1].strip())

            text = ''.join(processed_lines)

            return text
        except Exception as e:
            print(f"try_remove_newline error: {str(e)}")
            return text  # Return original text on error

    def _handle_text(self, text: str) -> str:
        """
        Process regular text content, preserving paragraph structure
        """
        try:
            if not text:
                return ""

            if text.strip().startswith("\\begin{array}") and text.strip().endswith("\\end{array}"):
                text = "$$" + text + "$$"
            elif ("_{" in text or "^{" in text or "\\" in text or "_ {" in text or "^ {" in text) and ("$" not in text) and ("\\begin" not in text):
                text = "$" + text + "$"

            # Process formulas in text before handling other text processing
            text = self._process_formulas_in_text(text)

            text = self.try_remove_newline(text)

            # Return processed text
            return text
        except Exception as e:
            print(f"_handle_text error: {str(e)}")
            return text  # Return original text on error

    def _process_formulas_in_text(self, text: str) -> str:
        """
        Process mathematical formulas in text by iteratively finding and replacing formulas.
        - Identify inline and block formulas
        - Replace newlines within formulas with \\
        """
        try:
            # Define formula delimiters and their corresponding patterns
            delimiters = [
                ('$$', '$$'),  # Block formula with $$
                ('\\[', '\\]'),  # Block formula with \[ \]
                ('$', '$'),  # Inline formula with $
                ('\\(', '\\)')  # Inline formula with \( \)
            ]

            # Process the text by iterating through each delimiter type
            result = text

            for start_delim, end_delim in delimiters:
                # Create a pattern that matches from start to end delimiter
                # Using a custom approach to avoid issues with nested delimiters
                current_pos = 0
                processed_parts = []

                while current_pos < len(result):
                    # Find the next start delimiter
                    start_pos = result.find(start_delim, current_pos)
                    if start_pos == -1:
                        # No more formulas of this type
                        processed_parts.append(result[current_pos:])
                        break

                    # Add text before the formula
                    processed_parts.append(result[current_pos:start_pos])

                    # Find the matching end delimiter
                    end_pos = result.find(end_delim, start_pos + len(start_delim))
                    if end_pos == -1:
                        # No matching end delimiter, treat as regular text
                        processed_parts.append(result[start_pos:])
                        break

                    # Extract the formula content (without delimiters)
                    formula_content = result[start_pos + len(start_delim):end_pos]

                    # Process the formula content - replace newlines with \\
                    processed_formula = formula_content.replace('\n', ' \\\\ ')

                    # Add the processed formula with its delimiters
                    processed_parts.append(f"{start_delim}{processed_formula}{end_delim}")

                    # Move past this formula
                    current_pos = end_pos + len(end_delim)

                # Update the result with processed text
                result = ''.join(processed_parts)
            return result
        except Exception as e:
            print(f"_process_formulas_in_text error: {str(e)}")
            return text  # Return original text on error

    def _remove_newline_in_heading(self, text: str) -> str:
        """
        Remove newline in heading
        """
        try:
            # Handle Chinese text line breaks
            def is_chinese(char):
                return '\u4e00' <= char <= '\u9fff'

            # Check if the text contains Chinese characters
            if any(is_chinese(char) for char in text):
                return text.replace('\n', '')
            else:
                return text.replace('\n', ' ')

        except Exception as e:
            print(f"_remove_newline_in_heading error: {str(e)}")
            return text

    def _handle_heading(self, text: str, label: str) -> str:
        """
        Convert section headings to appropriate markdown format
        """
        try:
            level = self.heading_levels.get(label, '#')
            text = text.strip()
            text = self._remove_newline_in_heading(text)
            text = self._handle_text(text)
            return f"{level} {text}\n\n"
        except Exception as e:
            print(f"_handle_heading error: {str(e)}")
            return f"# Error processing heading: {text}\n\n"

    def _handle_list_item(self, text: str) -> str:
        """
        Convert list items to markdown list format
        """
        try:
            return f"- {text.strip()}\n"
        except Exception as e:
            print(f"_handle_list_item error: {str(e)}")
            return f"- Error processing list item: {text}\n"

    def _handle_figure(self, text: str, section_count: int) -> str:
        """
        Handle figure content
        """
        try:
            # Check if it's a file path starting with "figures/"
            if text.startswith("figures/"):
                # Convert to relative path from markdown directory to figures directory
                relative_path = f"../{text}"
                return f"![Figure {section_count}]({relative_path})\n\n"

            # Check if it's already a markdown format image link
            if text.startswith("!["):
                # Already in markdown format, return directly
                return f"{text}\n\n"

            # If it's still base64 format, maintain original logic
            if text.startswith("data:image/"):
                return f"![Figure {section_count}]({text})\n\n"
            elif ";" in text and "," in text:
                return f"![Figure {section_count}]({text})\n\n"
            else:
                # Assume it's raw base64, convert to data URI
                img_format = "png"
                data_uri = f"data:image/{img_format};base64,{text}"
                return f"![Figure {section_count}]({data_uri})\n\n"

        except Exception as e:
            print(f"_handle_figure error: {str(e)}")
            return f"*[Error processing figure: {str(e)}]*\n\n"

    def _handle_table(self, text: str) -> str:
        """
        Convert table content to markdown format
        """
        try:
            markdown_content = []
            if '<table' in text.lower() or '<tr' in text.lower():
                markdown_table = extract_table_from_html(text)
                markdown_content.append(markdown_table + "\n")
            else:
                table_lines = text.split('\n')
                if table_lines:
                    col_count = len(table_lines[0].split()) if table_lines[0] else 1
                    header = '| ' + ' | '.join(table_lines[0].split()) + ' |'
                    markdown_content.append(header)
                    markdown_content.append('| ' + ' | '.join(['---'] * col_count) + ' |')
                    for line in table_lines[1:]:
                        cells = line.split()
                        while len(cells) < col_count:
                            cells.append('')
                        markdown_content.append('| ' + ' | '.join(cells) + ' |')
            return '\n'.join(markdown_content) + '\n\n'
        except Exception as e:
            print(f"_handle_table error: {str(e)}")
            return f"*[Error processing table: {str(e)}]*\n\n"

    def _handle_algorithm(self, text: str) -> str:
        """
        Process algorithm blocks with proper formatting
        """
        try:
            # Remove algorithm environment tags if present
            text = re.sub(r'\\begin\{algorithm\}(.*?)\\end\{algorithm\}', r'\1', text, flags=re.DOTALL)
            text = text.replace('\\begin{algorithm}', '').replace('\\end{algorithm}', '')
            text = text.replace('\\begin{algorithmic}', '').replace('\\end{algorithmic}', '')

            # Process the algorithm text
            lines = text.strip().split('\n')

            # Check if there's a caption or label
            caption = ""
            algorithm_text = []

            for line in lines:
                if '\\caption' in line:
                    # Extract caption text
                    caption_match = re.search(r'\\caption\{(.*?)\}', line)
                    if caption_match:
                        caption = f"**{caption_match.group(1)}**\n\n"
                    continue
                elif '\\label' in line:
                    continue  # Skip label lines
                else:
                    algorithm_text.append(line)

            # Join the algorithm text and wrap in code block
            formatted_text = '\n'.join(algorithm_text)

            # Return the formatted algorithm with caption
            return f"{caption}```\n{formatted_text}\n```\n\n"
        except Exception as e:
            print(f"_handle_algorithm error: {str(e)}")
            return f"*[Error processing algorithm: {str(e)}]*\n\n{text}\n\n"

    def _handle_formula(self, text: str) -> str:
        """
        Handle formula-specific content
        """
        try:
            # Process the formula content
            processed_text = self._process_formulas_in_text(text)

            # For formula blocks, ensure they're properly formatted in markdown
            if '$$' not in processed_text and '\\[' not in processed_text:
                # If no block formula delimiters are present, wrap in $$ for block formula
                processed_text = f'$${processed_text}$$'

            return f"{processed_text}\n\n"
        except Exception as e:
            print(f"_handle_formula error: {str(e)}")
            return f"*[Error processing formula: {str(e)}]*\n\n"

    def convert(self, recognition_results: List[Dict[str, Any]]) -> str:
        """
        Convert recognition results to markdown format
        """
        try:
            markdown_content = []

            for section_count, result in enumerate(recognition_results):
                try:
                    label = result.get('label', '')
                    text = result.get('text', '').strip()

                    # Skip empty text
                    if not text:
                        continue

                    # Handle different content types
                    if label in {'title', 'sec', 'sub_sec'}:
                        markdown_content.append(self._handle_heading(text, label))
                    elif label == 'list':
                        markdown_content.append(self._handle_list_item(text))
                    elif label == 'fig':
                        markdown_content.append(self._handle_figure(text, section_count))
                    elif label == 'tab':
                        markdown_content.append(self._handle_table(text))
                    elif label == 'alg':
                        markdown_content.append(self._handle_algorithm(text))
                    elif label == 'formula':
                        markdown_content.append(self._handle_formula(text))
                    elif label not in self.special_labels:
                        # Handle regular text (paragraphs, etc.)
                        processed_text = self._handle_text(text)
                        markdown_content.append(f"{processed_text}\n\n")
                except Exception as e:
                    print(f"Error processing item {section_count}: {str(e)}")
                    # Add a placeholder for the failed item
                    markdown_content.append(f"*[Error processing content]*\n\n")

            # Join all content and apply post-processing
            result = ''.join(markdown_content)
            return self._post_process(result)
        except Exception as e:
            print(f"convert error: {str(e)}")
            return f"Error generating markdown content: {str(e)}"

    def _post_process(self, markdown_content: str) -> str:
        """
        Apply post-processing fixes to the generated markdown content
        """
        try:
            # Handle author information
            author_pattern = re.compile(r'\\author\{(.*?)\}', re.DOTALL)

            def process_author_match(match):
                # Extract author content
                author_content = match.group(1)
                # Process the author content
                return self._handle_text(author_content)

            # Replace \author{...} with processed content
            markdown_content = author_pattern.sub(process_author_match, markdown_content)

            # Handle special case where author is inside math environment
            math_author_pattern = re.compile(r'\$(\\author\{.*?\})\$', re.DOTALL)
            match = math_author_pattern.search(markdown_content)
            if match:
                # Extract the author command
                author_cmd = match.group(1)
                # Extract content from author command
                author_content_match = re.search(r'\\author\{(.*?)\}', author_cmd, re.DOTALL)
                if author_content_match:
                    # Get author content and process it
                    author_content = author_content_match.group(1)
                    processed_content = self._handle_text(author_content)
                    # Replace the entire $\author{...}$ block with processed content
                    markdown_content = markdown_content.replace(match.group(0), processed_content)

            # Replace LaTeX abstract environment with plain text
            markdown_content = re.sub(r'\\begin\{abstract\}(.*?)\\end\{abstract\}',
                                     r'**Abstract** \1',
                                     markdown_content,
                                     flags=re.DOTALL)

            # Replace standalone \begin{abstract} (without matching end)
            markdown_content = re.sub(r'\\begin\{abstract\}',
                                     r'**Abstract**',
                                     markdown_content)

            # Replace LaTeX equation numbers with tag format, handling cases with extra backslashes
            markdown_content = re.sub(r'\\eqno\{\((.*?)\)\}',
                                    r'\\tag{\1}',
                                    markdown_content)

            # Find the starting tag of the formula
            markdown_content = markdown_content.replace("\[ \\\\", "$$ \\\\")

            # Find the ending tag of the formula (ensure this is the only ending tag)
            markdown_content = markdown_content.replace("\\\\ \]", "\\\\ $$")

            # Fix other common LaTeX issues
            replacements = [
                # Fix spacing issues in subscripts and superscripts
                (r'_ {', r'_{'),
                (r'^ {', r'^{'),

                # Fix potential issues with multiple consecutive newlines
                (r'\n{3,}', r'\n\n')
            ]

            for old, new in replacements:
                markdown_content = re.sub(old, new, markdown_content)

            return markdown_content
        except Exception as e:
            print(f"_post_process error: {str(e)}")
            return markdown_content  # Return original content if post-processing fails


In [ ]:
"""
Copyright (c) 2025 Bytedance Ltd. and/or its affiliates
SPDX-License-Identifier: MIT
"""

import copy
import json
import os
import re
from dataclasses import dataclass
from typing import List, Tuple

import albumentations as alb
import cv2
import numpy as np
from albumentations.pytorch import ToTensorV2
from PIL import Image
from timm.data.constants import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD
from torchvision.transforms.functional import resize




def save_figure_to_local(pil_crop, save_dir, image_name, reading_order):
    """Save cropped figure to local file system

    Args:
        pil_crop: PIL Image object of the cropped figure
        save_dir: Base directory to save results
        image_name: Name of the source image/document
        reading_order: Reading order of the figure in the document

    Returns:
        str: Filename of the saved figure
    """
    try:
        # Create figures directory if it doesn't exist
        figures_dir = os.path.join(save_dir, "markdown", "figures")
        # os.makedirs(figures_dir, exist_ok=True)

        # Generate figure filename
        figure_filename = f"{image_name}_figure_{reading_order:03d}.png"
        figure_path = os.path.join(figures_dir, figure_filename)

        # Save the figure
        pil_crop.save(figure_path, format="PNG", quality=95)

        # print(f"Saved figure: {figure_filename}")
        return figure_filename

    except Exception as e:
        print(f"Error saving figure: {str(e)}")
        # Return a fallback filename
        return f"{image_name}_figure_{reading_order:03d}_error.png"


def convert_pdf_to_images(pdf_path, target_size=896):
    """Convert PDF pages to images

    Args:
        pdf_path: Path to PDF file
        target_size: Target size for the longest dimension

    Returns:
        List of PIL Images
    """
    images = []
    try:
        doc = pymupdf.open(pdf_path)

        for page_num in range(len(doc)):
            page = doc[page_num]

            # Calculate scale to make longest dimension equal to target_size
            rect = page.rect
            scale = target_size / max(rect.width, rect.height)

            # Render page as image
            mat = pymupdf.Matrix(scale, scale)
            pix = page.get_pixmap(matrix=mat)

            # Convert to PIL Image
            img_data = pix.tobytes("png")
            pil_image = Image.open(io.BytesIO(img_data))
            images.append(pil_image)

        doc.close()
        print(f"Successfully converted {len(images)} pages from PDF")
        return images

    except Exception as e:
        print(f"Error converting PDF to images: {str(e)}")
        return []


def is_pdf_file(file_path):
    """Check if file is a PDF"""
    return file_path.lower().endswith('.pdf')


def save_combined_pdf_results(all_page_results, pdf_path, save_dir):
    """Save combined results for multi-page PDF with both JSON and Markdown

    Args:
        all_page_results: List of results for all pages
        pdf_path: Path to original PDF file
        save_dir: Directory to save results

    Returns:
        Path to saved combined JSON file
    """
    # Create output filename based on PDF name
    base_name = os.path.splitext(os.path.basename(pdf_path))[0]

    # Prepare combined results
    combined_results = {
        "source_file": pdf_path,
        "total_pages": len(all_page_results),
        "pages": all_page_results
    }

    # Save combined JSON results
    json_filename = f"{base_name}.json"
    json_path = os.path.join(save_dir, "recognition_json", json_filename)
    os.makedirs(os.path.dirname(json_path), exist_ok=True)

    with open(json_path, 'w', encoding='utf-8') as f:
        json.dump(combined_results, f, indent=2, ensure_ascii=False)

    # Generate and save combined markdown
    try:
        markdown_converter = MarkdownConverter()

        # Combine all page results into a single list for markdown conversion
        all_elements = []
        for page_data in all_page_results:
            page_elements = page_data.get("elements", [])
            if page_elements:
                # Add page separator if not the first page
                if all_elements:
                    all_elements.append({
                        "label": "page_separator",
                        "text": f"\n\n---\n\n",
                        "reading_order": len(all_elements)
                    })
                all_elements.extend(page_elements)

        # Generate markdown content
        markdown_content = markdown_converter.convert(all_elements)

        # Save markdown file
        markdown_filename = f"{base_name}.md"
        markdown_path = os.path.join(save_dir, "markdown", markdown_filename)
        os.makedirs(os.path.dirname(markdown_path), exist_ok=True)

        with open(markdown_path, 'w', encoding='utf-8') as f:
            f.write(markdown_content)

        # print(f"Combined markdown saved to: {markdown_path}")

    except ImportError:
        print("MarkdownConverter not available, skipping markdown generation")
    except Exception as e:
        print(f"Error generating markdown: {e}")

    # print(f"Combined JSON results saved to: {json_path}")
    return json_path


def alb_wrapper(transform):
    def f(im):
        return transform(image=np.asarray(im))["image"]

    return f


test_transform = alb_wrapper(
    alb.Compose(
        [
            alb.Normalize(IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD),
            ToTensorV2(),
        ]
    )
)


def check_coord_valid(x1, y1, x2, y2, image_size=None, abs_coord=True):
    # print(f"check_coord_valid: {x1}, {y1}, {x2}, {y2}, {image_size}, {abs_coord}")
    if x2 <= x1 or y2 <= y1:
        return False, f"[{x1}, {y1}, {x2}, {y2}]"
    if x1 < 0 or y1 < 0:
        return False, f"[{x1}, {y1}, {x2}, {y2}]"
    if not abs_coord:
        if x2 > 1 or y2 > 1:
            return False, f"[{x1}, {y1}, {x2}, {y2}]"
    elif image_size is not None: # has image size
        if x2 > image_size[0] or y2 > image_size[1]:
            return False, f"[{x1}, {y1}, {x2}, {y2}]"
    return True, None


def adjust_box_edges(image, boxes: List[List[float]], max_pixels=15, threshold=0.2):
    """
    Image: cv2.image object, or Path
    Input: boxes: list of boxes [[x1, y1, x2, y2]]. Using absolute coordinates.
    """
    if isinstance(image, str):
        image = cv2.imread(image)
    img_h, img_w = image.shape[:2]
    new_boxes = []
    for box in boxes:
        best_box = copy.deepcopy(box)

        def check_edge(img, current_box, i, is_vertical):
            edge = current_box[i]
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            _, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

            if is_vertical:
                line = binary[current_box[1] : current_box[3] + 1, edge]
            else:
                line = binary[edge, current_box[0] : current_box[2] + 1]

            transitions = np.abs(np.diff(line))
            return np.sum(transitions) / len(transitions)

        # Only widen the box
        edges = [(0, -1, True), (2, 1, True), (1, -1, False), (3, 1, False)]

        current_box = copy.deepcopy(box)
        # make sure the box is within the image
        current_box[0] = min(max(current_box[0], 0), img_w - 1)
        current_box[1] = min(max(current_box[1], 0), img_h - 1)
        current_box[2] = min(max(current_box[2], 0), img_w - 1)
        current_box[3] = min(max(current_box[3], 0), img_h - 1)

        for i, direction, is_vertical in edges:
            best_score = check_edge(image, current_box, i, is_vertical)
            if best_score <= threshold:
                continue
            for step in range(max_pixels):
                current_box[i] += direction
                if i == 0 or i == 2:
                    current_box[i] = min(max(current_box[i], 0), img_w - 1)
                else:
                    current_box[i] = min(max(current_box[i], 0), img_h - 1)
                score = check_edge(image, current_box, i, is_vertical)

                if score < best_score:
                    best_score = score
                    best_box = copy.deepcopy(current_box)

                if score <= threshold:
                    break
        new_boxes.append(best_box)

    return new_boxes


def parse_layout_string(bbox_str):
    """Parse layout string using regular expressions"""
    pattern = r"\[(\d*\.?\d+),\s*(\d*\.?\d+),\s*(\d*\.?\d+),\s*(\d*\.?\d+)\]\s*(\w+)"
    matches = re.finditer(pattern, bbox_str)

    parsed_results = []
    for match in matches:
        coords = [float(match.group(i)) for i in range(1, 5)]
        label = match.group(5).strip()
        parsed_results.append((coords, label))

    return parsed_results


@dataclass
class ImageDimensions:
    """Class to store image dimensions"""
    original_w: int
    original_h: int
    padded_w: int
    padded_h: int


def map_to_original_coordinates(x1, y1, x2, y2, dims: ImageDimensions) -> Tuple[int, int, int, int]:
    """Map coordinates from padded image back to original image

    Args:
        x1, y1, x2, y2: Coordinates in padded image
        dims: Image dimensions object

    Returns:
        tuple: (x1, y1, x2, y2) coordinates in original image
    """
    try:
        # Calculate padding offsets
        top = (dims.padded_h - dims.original_h) // 2
        left = (dims.padded_w - dims.original_w) // 2

        # Map back to original coordinates
        orig_x1 = max(0, x1 - left)
        orig_y1 = max(0, y1 - top)
        orig_x2 = min(dims.original_w, x2 - left)
        orig_y2 = min(dims.original_h, y2 - top)

        # Ensure we have a valid box (width and height > 0)
        if orig_x2 <= orig_x1:
            orig_x2 = min(orig_x1 + 1, dims.original_w)
        if orig_y2 <= orig_y1:
            orig_y2 = min(orig_y1 + 1, dims.original_h)

        return int(orig_x1), int(orig_y1), int(orig_x2), int(orig_y2)
    except Exception as e:
        print(f"map_to_original_coordinates error: {str(e)}")
        # Return safe coordinates
        return 0, 0, min(100, dims.original_w), min(100, dims.original_h)


def map_to_relevant_coordinates(abs_coords, dims: ImageDimensions):
    """
        From absolute coordinates to relevant coordinates
        e.g. [100, 100, 200, 200] -> [0.1, 0.2, 0.3, 0.4]
    """
    try:
        x1, y1, x2, y2 = abs_coords
        return round(x1 / dims.original_w, 3), round(y1 / dims.original_h, 3), round(x2 / dims.original_w, 3), round(y2 / dims.original_h, 3)
    except Exception as e:
        print(f"map_to_relevant_coordinates error: {str(e)}")
        return 0.0, 0.0, 1.0, 1.0  # Return full image coordinates


def process_coordinates(coords, padded_image, dims: ImageDimensions, previous_box=None):
    """Process and adjust coordinates

    Args:
        coords: Normalized coordinates [x1, y1, x2, y2]
        padded_image: Padded image
        dims: Image dimensions object
        previous_box: Previous box coordinates for overlap adjustment

    Returns:
        tuple: (x1, y1, x2, y2, orig_x1, orig_y1, orig_x2, orig_y2, new_previous_box)
    """
    try:
        # Convert normalized coordinates to absolute coordinates
        x1, y1 = int(coords[0] * dims.padded_w), int(coords[1] * dims.padded_h)
        x2, y2 = int(coords[2] * dims.padded_w), int(coords[3] * dims.padded_h)

        # Ensure coordinates are within image bounds before adjustment
        x1 = max(0, min(x1, dims.padded_w - 1))
        y1 = max(0, min(y1, dims.padded_h - 1))
        x2 = max(0, min(x2, dims.padded_w))
        y2 = max(0, min(y2, dims.padded_h))

        # Ensure width and height are at least 1 pixel
        if x2 <= x1:
            x2 = min(x1 + 1, dims.padded_w)
        if y2 <= y1:
            y2 = min(y1 + 1, dims.padded_h)

        # Extend box boundaries
        new_boxes = adjust_box_edges(padded_image, [[x1, y1, x2, y2]])
        x1, y1, x2, y2 = new_boxes[0]

        # Ensure coordinates are still within image bounds after adjustment
        x1 = max(0, min(x1, dims.padded_w - 1))
        y1 = max(0, min(y1, dims.padded_h - 1))
        x2 = max(0, min(x2, dims.padded_w))
        y2 = max(0, min(y2, dims.padded_h))

        # Ensure width and height are at least 1 pixel after adjustment
        if x2 <= x1:
            x2 = min(x1 + 1, dims.padded_w)
        if y2 <= y1:
            y2 = min(y1 + 1, dims.padded_h)

        # Check for overlap with previous box and adjust
        if previous_box is not None:
            prev_x1, prev_y1, prev_x2, prev_y2 = previous_box
            if (x1 < prev_x2 and x2 > prev_x1) and (y1 < prev_y2 and y2 > prev_y1):
                y1 = prev_y2
                # Ensure y1 is still valid
                y1 = min(y1, dims.padded_h - 1)
                # Make sure y2 is still greater than y1
                if y2 <= y1:
                    y2 = min(y1 + 1, dims.padded_h)

        # Update previous box
        new_previous_box = [x1, y1, x2, y2]

        # Map to original coordinates
        orig_x1, orig_y1, orig_x2, orig_y2 = map_to_original_coordinates(
            x1, y1, x2, y2, dims
        )

        return x1, y1, x2, y2, orig_x1, orig_y1, orig_x2, orig_y2, new_previous_box
    except Exception as e:
        print(f"process_coordinates error: {str(e)}")
        # Return safe values
        orig_x1, orig_y1, orig_x2, orig_y2 = 0, 0, min(100, dims.original_w), min(100, dims.original_h)
        return 0, 0, 100, 100, orig_x1, orig_y1, orig_x2, orig_y2, [0, 0, 100, 100]


def prepare_image(image) -> Tuple[np.ndarray, ImageDimensions]:
    """Load and prepare image with padding while maintaining aspect ratio

    Args:
        image: PIL image

    Returns:
        tuple: (padded_image, image_dimensions)
    """
    try:
        # Convert PIL image to OpenCV format
        image = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
        original_h, original_w = image.shape[:2]

        # Calculate padding to make square image
        max_size = max(original_h, original_w)
        top = (max_size - original_h) // 2
        bottom = max_size - original_h - top
        left = (max_size - original_w) // 2
        right = max_size - original_w - left

        # Apply padding
        padded_image = cv2.copyMakeBorder(image, top, bottom, left, right,
                                cv2.BORDER_CONSTANT, value=(0, 0, 0))

        padded_h, padded_w = padded_image.shape[:2]

        dimensions = ImageDimensions(
            original_w=original_w,
            original_h=original_h,
            padded_w=padded_w,
            padded_h=padded_h
        )

        return padded_image, dimensions
    except Exception as e:
        print(f"prepare_image error: {str(e)}")
        # Create a minimal valid image and dimensions
        h, w = image.height, image.width
        dimensions = ImageDimensions(
            original_w=w,
            original_h=h,
            padded_w=w,
            padded_h=h
        )
        # Return a black image of the same size
        return np.zeros((h, w, 3), dtype=np.uint8), dimensions


def setup_output_dirs(save_dir):
    """Create necessary output directories"""
    os.makedirs(save_dir, exist_ok=True)
    os.makedirs(os.path.join(save_dir, "markdown"), exist_ok=True)
    os.makedirs(os.path.join(save_dir, "recognition_json"), exist_ok=True)
    os.makedirs(os.path.join(save_dir, "markdown", "figures"), exist_ok=True)


def save_outputs(recognition_results, image_path, save_dir):
    """Save JSON and markdown outputs"""
    basename = os.path.splitext(os.path.basename(image_path))[0]

    # Save JSON file
    json_path = os.path.join(save_dir, "recognition_json", f"{basename}.json")
    with open(json_path, "w", encoding="utf-8") as f:
        json.dump(recognition_results, f, ensure_ascii=False, indent=2)

    # Generate and save markdown file
    markdown_converter = MarkdownConverter()
    markdown_content = markdown_converter.convert(recognition_results)
    markdown_path = os.path.join(save_dir, "markdown", f"{basename}.md")
    with open(markdown_path, "w", encoding="utf-8") as f:
        f.write(markdown_content)

    return json_path


def crop_margin(img: Image.Image) -> Image.Image:
    """Crop margins from image"""
    try:
        width, height = img.size
        if width == 0 or height == 0:
            print("Warning: Image has zero width or height")
            return img

        data = np.array(img.convert("L"))
        data = data.astype(np.uint8)
        max_val = data.max()
        min_val = data.min()
        if max_val == min_val:
            return img
        data = (data - min_val) / (max_val - min_val) * 255
        gray = 255 * (data < 200).astype(np.uint8)

        coords = cv2.findNonZero(gray)  # Find all non-zero points (text)
        if coords is None:
            return img
        a, b, w, h = cv2.boundingRect(coords)  # Find minimum spanning bounding box

        # Ensure crop coordinates are within image bounds
        a = max(0, a)
        b = max(0, b)
        w = min(w, width - a)
        h = min(h, height - b)

        # Only crop if we have a valid region
        if w > 0 and h > 0:
            return img.crop((a, b, a + w, b + h))
        return img
    except Exception as e:
        print(f"crop_margin error: {str(e)}")
        return img  # Return original image on error

In [ ]:
import glob
import os
import json
import base64
import re
from io import BytesIO
import numpy as np

import cv2
import torch
from PIL import Image
from transformers import AutoProcessor, VisionEncoderDecoderModel

# Configuration - Edit these variables instead of using command line arguments
MODEL_PATH = "./hf_model"  # Path to Hugging Face model
INPUT_PATH = "/flow_chart.png"      # Path to input image/PDF or directory of files
SAVE_DIR = "/media/test"         # Directory to save parsing results (None = same as input directory)
MAX_BATCH_SIZE = 16       # Maximum number of document elements to parse in a single batch

class DOLPHIN:
    def __init__(self, model_id_or_path):
        """Initialize the Hugging Face model

        Args:
            model_id_or_path: Path to local model or Hugging Face model ID
        """
        # Load model from local path or Hugging Face hub
        self.processor = AutoProcessor.from_pretrained("ByteDance/Dolphin")
        self.model = VisionEncoderDecoderModel.from_pretrained("ByteDance/Dolphin")
        self.model.eval()

        # Set device and precision
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model.to(self.device)
        if self.device == "cuda":
            self.model = self.model.half()  # Use half precision only on GPU

        # set tokenizer
        self.tokenizer = self.processor.tokenizer

    def chat(self, prompt, image):
        """Process an image or batch of images with the given prompt(s)

        Args:
            prompt: Text prompt or list of prompts to guide the model
            image: PIL Image or list of PIL Images to process

        Returns:
            Generated text or list of texts from the model
        """
        # Check if we're dealing with a batch
        is_batch = isinstance(image, list)

        if not is_batch:
            # Single image, wrap it in a list for consistent processing
            images = [image]
            prompts = [prompt]
        else:
            # Batch of images
            images = image
            prompts = prompt if isinstance(prompt, list) else [prompt] * len(images)

        # Prepare image
        batch_inputs = self.processor(images, return_tensors="pt", padding=True)
        batch_pixel_values = batch_inputs.pixel_values.to(self.device)
        if self.device == "cuda":
            batch_pixel_values = batch_pixel_values.half()

        # Prepare prompt
        prompts = [f"<s>{p} <Answer/>" for p in prompts]
        batch_prompt_inputs = self.tokenizer(
            prompts,
            add_special_tokens=False,
            return_tensors="pt"
        )

        batch_prompt_ids = batch_prompt_inputs.input_ids.to(self.device)
        batch_attention_mask = batch_prompt_inputs.attention_mask.to(self.device)

        # Generate text
        outputs = self.model.generate(
            pixel_values=batch_pixel_values,
            decoder_input_ids=batch_prompt_ids,
            decoder_attention_mask=batch_attention_mask,
            min_length=1,
            max_length=4096,
            pad_token_id=self.tokenizer.pad_token_id,
            eos_token_id=self.tokenizer.eos_token_id,
            use_cache=True,
            bad_words_ids=[[self.tokenizer.unk_token_id]],
            return_dict_in_generate=True,
            do_sample=False,
            num_beams=1,
            repetition_penalty=1.1
        )

        # Process output
        sequences = self.tokenizer.batch_decode(outputs.sequences, skip_special_tokens=False)

        # Clean prompt text from output
        results = []
        for i, sequence in enumerate(sequences):
            cleaned = sequence.replace(prompts[i], "").replace("<pad>", "").replace("</s>", "").strip()
            results.append(cleaned)

        # Return a single result for single image input
        if not is_batch:
            return results[0]
        return results

    def parse_layout(self, image):
        """Parse document layout to get element positions and types

        Args:
            image: PIL Image object

        Returns:
            String containing layout information in token format
        """
        # Use the correct prompt for layout analysis
        layout_prompt = "Parse the reading order of this document."
        return self.chat(layout_prompt, image)

    def extract_text(self, image):
        """Extract readable text from image

        Args:
            image: PIL Image object

        Returns:
            String containing extracted text
        """
        # Use specific prompts for text extraction
        text_prompt = "Extract all text content from this image in reading order."
        return self.chat(text_prompt, image)

    def extract_table(self, image):
        """Extract table content from image

        Args:
            image: PIL Image object

        Returns:
            String containing table content
        """
        # Use specific prompt for table extraction
        table_prompt = "Parse the table content from this image."
        return self.chat(table_prompt, image)

    def summarize_image(self, image, summary_type="general"):
        """Summarize an image with different types of analysis

        Args:
            image: PIL Image object
            summary_type: Type of summary ("general", "detailed", "technical", "content")

        Returns:
            String summary of the image
        """
        summary_prompts = {
            "general": "Provide a brief summary of what you see in this image.",
            "detailed": "Describe this image in detail, including all visible elements, colors, layout, and any text or objects present.",
            "technical": "Analyze this image from a technical perspective, describing its structure, layout, design elements, and any technical content.",
            "content": "Extract and summarize all the textual and informational content from this image.",
            "flowchart": "Describe the flow and logic of this flowchart or diagram, explaining the process step by step.",
            "document": "Summarize the document structure and main content of this image.",
            "text_extraction": "Extract all text content from this image in reading order, preserving structure and formatting.",
            "markdown": "Convert this document image to markdown format, preserving structure and formatting."
        }

        prompt = summary_prompts.get(summary_type, summary_prompts["general"])
        return self.chat(prompt, image)


def parse_layout_tokens(layout_string):
    """Parse layout string to extract bounding boxes and labels

    Args:
        layout_string: String containing layout tokens

    Returns:
        List of tuples (bbox, label) where bbox is [x1, y1, x2, y2]
    """
    elements = []

    # Pattern to match coordinate pairs and labels
    # Looking for patterns like [0.10,0.27,0.90,0.58] fig[PAIR_SEP]
    pattern = r'\[([0-9.]+),([0-9.]+),([0-9.]+),([0-9.]+)\]\s*([a-zA-Z]+)'

    matches = re.findall(pattern, layout_string)

    for match in matches:
        x1, y1, x2, y2, label = match
        bbox = [float(x1), float(y1), float(x2), float(y2)]
        elements.append((bbox, label))

    return elements


def convert_normalized_coords(bbox, image_width, image_height):
    """Convert normalized coordinates to pixel coordinates

    Args:
        bbox: [x1, y1, x2, y2] in normalized coordinates (0-1)
        image_width: Width of image in pixels
        image_height: Height of image in pixels

    Returns:
        [x1, y1, x2, y2] in pixel coordinates
    """
    x1, y1, x2, y2 = bbox

    # Convert to pixel coordinates
    pixel_x1 = int(x1 * image_width)
    pixel_y1 = int(y1 * image_height)
    pixel_x2 = int(x2 * image_width)
    pixel_y2 = int(y2 * image_height)

    return [pixel_x1, pixel_y1, pixel_x2, pixel_y2]


def process_single_image_fixed(image, model, save_dir, image_name, max_batch_size=None, save_individual=True):
    """Process a single image with proper two-stage approach

    Args:
        image: PIL Image object
        model: DOLPHIN model instance
        save_dir: Directory to save results
        image_name: Name for the output file
        max_batch_size: Maximum batch size for processing
        save_individual: Whether to save individual results

    Returns:
        Tuple of (json_path, recognition_results)
    """

    # Stage 1: Layout Analysis - Get element positions and types
    print("Stage 1: Analyzing document layout...")
    layout_output = model.parse_layout(image)

    # Parse layout tokens if they exist
    layout_elements = parse_layout_tokens(layout_output)

    recognition_results = []

    if layout_elements:
        print(f"Found {len(layout_elements)} layout elements")

        # Stage 2: Content Extraction for each element
        print("Stage 2: Extracting content from elements...")

        for i, (bbox, label) in enumerate(layout_elements):
            try:
                # Convert normalized coordinates to pixel coordinates
                pixel_bbox = convert_normalized_coords(bbox, image.width, image.height)
                x1, y1, x2, y2 = pixel_bbox

                # Crop the element from the image
                cropped_image = image.crop((x1, y1, x2, y2))

                # Extract content based on element type
                if label == "tab":  # Table
                    content = model.extract_table(cropped_image)
                elif label == "fig":  # Figure
                    content = f"[Figure: {cropped_image.size[0]}x{cropped_image.size[1]}]"
                    # Save figure
                    if save_dir:
                        fig_path = os.path.join(save_dir, "figures", f"{image_name}_fig_{i:03d}.png")
                        os.makedirs(os.path.dirname(fig_path), exist_ok=True)
                        cropped_image.save(fig_path)
                        content += f" Saved to: {fig_path}"
                else:  # Text elements (para, sec, foot, etc.)
                    content = model.extract_text(cropped_image)

                recognition_results.append({
                    "element_id": i,
                    "label": label,
                    "bbox": bbox,  # Normalized coordinates
                    "pixel_bbox": pixel_bbox,  # Pixel coordinates
                    "text": content.strip() if content else "",
                    "reading_order": i
                })

            except Exception as e:
                print(f"Error processing element {i} ({label}): {e}")
                continue

    else:
        # Fallback: If no layout elements found, extract text from entire image
        print("No layout elements detected, extracting text from entire image...")
        content = model.extract_text(image)

        recognition_results.append({
            "element_id": 0,
            "label": "document",
            "bbox": [0.0, 0.0, 1.0, 1.0],
            "pixel_bbox": [0, 0, image.width, image.height],
            "text": content.strip() if content else "",
            "reading_order": 0
        })

    # Save results
    json_path = None
    if save_individual and save_dir:
        os.makedirs(save_dir, exist_ok=True)
        json_filename = f"{image_name}_results.json"
        json_path = os.path.join(save_dir, json_filename)

        output_data = {
            "image_name": image_name,
            "image_size": {"width": image.width, "height": image.height},
            "layout_output": layout_output,
            "elements": recognition_results
        }

        with open(json_path, 'w', encoding='utf-8') as f:
            json.dump(output_data, f, indent=2, ensure_ascii=False)

    return json_path, recognition_results


def run_improved_document_parsing(model_path=MODEL_PATH, input_path=INPUT_PATH, save_dir=SAVE_DIR):
    """Improved document parsing with proper two-stage approach"""

    # Load Model
    print(f"Loading DOLPHIN model from: {model_path}")
    model = DOLPHIN(model_path)
    print("Model loaded successfully!")

    # Handle input
    if os.path.isfile(input_path):
        document_files = [input_path]
    else:
        # Directory of files
        file_extensions = [".jpg", ".jpeg", ".png", ".JPG", ".JPEG", ".PNG"]
        document_files = []
        for ext in file_extensions:
            document_files.extend(glob.glob(os.path.join(input_path, f"*{ext}")))
        document_files = sorted(document_files)

    if not document_files:
        print(f"No supported image files found in {input_path}")
        return []

    # Setup output directory
    if save_dir:
        os.makedirs(save_dir, exist_ok=True)
        os.makedirs(os.path.join(save_dir, "figures"), exist_ok=True)

    print(f"Processing {len(document_files)} files...")

    results = []
    for file_path in document_files:
        print(f"\n=== Processing: {os.path.basename(file_path)} ===")

        try:
            # Load image
            image = Image.open(file_path).convert("RGB")
            base_name = os.path.splitext(os.path.basename(file_path))[0]

            # Process with improved method
            json_path, recognition_results = process_single_image_fixed(
                image, model, save_dir, base_name
            )

            # Print readable results
            print(f"Extracted {len(recognition_results)} elements:")
            for result in recognition_results:
                print(f"  - {result['label']}: {result['text'][:100]}..." if len(result['text']) > 100 else f"  - {result['label']}: {result['text']}")

            results.append({
                'file': file_path,
                'json_path': json_path,
                'results': recognition_results
            })

        except Exception as e:
            print(f"Error processing {file_path}: {e}")
            continue

    return results


def run_text_extraction_only(input_path, save_dir=None, summary_type="text_extraction"):
    """Simple text extraction without layout analysis"""

    print("Loading DOLPHIN model for text extraction...")
    model = DOLPHIN(MODEL_PATH)
    print("Model loaded successfully!")

    if os.path.isfile(input_path):
        files = [input_path]
    else:
        file_extensions = [".jpg", ".jpeg", ".png", ".JPG", ".JPEG", ".PNG"]
        files = []
        for ext in file_extensions:
            files.extend(glob.glob(os.path.join(input_path, f"*{ext}")))
        files = sorted(files)

    results = {}

    for file_path in files:
        try:
            print(f"Processing: {os.path.basename(file_path)}")
            image = Image.open(file_path).convert("RGB")

            # Direct text extraction
            extracted_text = model.summarize_image(image, summary_type)
            results[os.path.basename(file_path)] = extracted_text

            print(f"Extracted text: {extracted_text[:200]}..." if len(extracted_text) > 200 else f"Extracted text: {extracted_text}")

        except Exception as e:
            print(f"Error processing {file_path}: {e}")
            results[os.path.basename(file_path)] = f"Error: {str(e)}"

    # Save if requested
    if save_dir and results:
        os.makedirs(save_dir, exist_ok=True)
        output_file = os.path.join(save_dir, "extracted_text.json")
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(results, f, indent=2, ensure_ascii=False)
        print(f"Results saved to: {output_file}")

    return results


# Example usage
if __name__ == "__main__":
    print("=== DOLPHIN Document Parser (Fixed Version) ===")

    # Option 1: Full document parsing with layout analysis
    print("\n1. Running full document parsing...")
    results = run_improved_document_parsing(
        input_path='/media/train',
        save_dir='/media/test'
    )

    # Option 2: Simple text extraction only
    print("\n2. Running simple text extraction...")
    text_results = run_text_extraction_only(
        input_path='/media/train',
        save_dir='/media/test',
        summary_type='text_extraction'
    )

    print("\nProcessing complete!")

=== DOLPHIN Document Parser (Fixed Version) ===

1. Running full document parsing...
Loading DOLPHIN model from: ./hf_model
Model loaded successfully!
Processing 2 files...

=== Processing: diagram.png ===
Stage 1: Analyzing document layout...
Found 18 layout elements
Stage 2: Extracting content from elements...
Extracted 18 elements:
  - fig: [Figure: 226x63] Saved to: /media/test/figures/diagram_fig_000.png
  - cap: [0.49,0.86,0.51,0.87] foot
  - para: [0.39,0.49,0.53,0.50] Text Block
  - sec: [0.46,0.03,0.59,0.05] header[PAIR_SEP][0.71,0.03,0.82,0.05] header[PAIR_SEP][0.21,0.07,0.82,0.13] pa...
  - sub: [0.46,0.03,0.52,0.04] header[PAIR_SEP][0.71,0.03,0.80,0.04] header[PAIR_SEP][0.29,0.07,0.80,0.13] pa...
  - para: [0.43,0.05,0.57,0.06] header[PAIR_SEP][0.61,0.05,0.82,0.06] header[PAIR_SEP][0.43,0.09,0.57,0.10] se...
  - para: [0.39,0.41,0.61,0.52] Figure
[0.38,0.52,0.61,0.57] Text Block
  - foot: [0.48,0.03,0.52,0.04] header[PAIR_SEP][0.71,0.03,0.73,0.04] header[PAIR_SEP][0.26,0.09

In [ ]:
import os
from google.colab import files

# Define the path to the folder you want to download
folder_path = '/media/train'

# Define the name for the zip file
zip_filename = 'test_folder.zip'

# Check if the folder exists
if not os.path.exists(folder_path):
    print(f"Error: Folder '{folder_path}' does not exist.")
    print("Please double-check the path. Most Colab data is in /content/.")
else:
    print(f"Attempting to zip and download: {folder_path}")
    # Create a zip archive of the folder
    # The '!' executes a shell command
    !zip -r {zip_filename} {folder_path}

    # Download the zip file
    files.download(zip_filename)
    print(f"Folder '{folder_path}' has been zipped to '{zip_filename}' and downloaded.")

Attempting to zip and download: /media/train
  adding: media/train/ (stored 0%)
  adding: media/train/flow_chart.png (deflated 2%)
  adding: media/train/diagram.png (deflated 1%)
  adding: media/train/markdown/ (stored 0%)
  adding: media/train/markdown/flow_chart.md (deflated 56%)
  adding: media/train/markdown/figures/ (stored 0%)
  adding: media/train/markdown/figures/flow_chart_figure_002.png (deflated 1%)
  adding: media/train/markdown/figures/flow_chart_figure_001.png (deflated 1%)
  adding: media/train/markdown/figures/flow_chart_figure_000.png (stored 0%)
  adding: media/train/recognition_json/ (stored 0%)
  adding: media/train/recognition_json/flow_chart.json (deflated 74%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Folder '/media/train' has been zipped to 'test_folder.zip' and downloaded.
